In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import joblib

warnings.filterwarnings('ignore')

# Настройка визуал
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print(" Все библиотеки загружены успешно!")


In [ ]:

INPUT_PATH = '../data/processed/transactions_with_features.csv'
OUTPUT_PATH = '../data/processed/transactions_with_features_final.csv'
PIPELINE_PATH = '../model/preprocessing_pipeline.pkl'


df = pd.read_csv(INPUT_PATH, low_memory=False)
print(f" Данные загружены: {df.shape}")
print(f"Колонок: {len(df.columns)}")
print(f"Строк: {len(df)}")

# Проверка целевой переменной
if 'target' in df.columns:
    print(f"\n Распределение классов:")
    print(df['target'].value_counts())
    print(f"Дисбаланс: {df['target'].value_counts()[0] / df['target'].value_counts()[1]:.2f}:1")
else:
    raise ValueError(" Колонка 'target' не найдена!")


In [ ]:


# Находим категориальные признаки
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
print(f"\nНайдено категориальных колонок: {len(categorical_cols)}")
if len(categorical_cols) > 0:
    print(f"Категориальные колонки: {categorical_cols}")

# Обработка категориальных признаков
label_encoders = {}
frequency_encodings = {}

for col in categorical_cols:
    if col == 'target' or col == 'user_id':
        continue
    
    print(f"\n--- Обработка колонки: {col} ---")
    
    # Проверяем количество уникальных значений
    unique_count = df[col].nunique()
    print(f"  Уникальных значений: {unique_count}")
    
    # Если много уникальных значений (>50) - используем frequency encoding
    # Если мало - можно использовать label encoding или one-hot
    if unique_count > 50:
        # Frequency Encoding для колонок с большим количеством уникальных значений
        freq_map = df[col].value_counts() / len(df)
        df[f'{col}_freq_encoding'] = df[col].map(freq_map)
        frequency_encodings[col] = freq_map
        print(f"   Создан frequency encoding: {col}_freq_encoding")
        # Удаляем исходную колонку
        df.drop(columns=[col], inplace=True)
    elif unique_count > 2:
        # Label Encoding для колонок с небольшим количеством уникальных значений
        le = LabelEncoder()
        df[f'{col}_encoded'] = le.fit_transform(df[col].astype(str).fillna('UNKNOWN'))
        label_encoders[col] = le
        print(f"   Создан label encoding: {col}_encoded")
        # Удаляем исходную колонку
        df.drop(columns=[col], inplace=True)
    else:
        # Бинарные признаки - можно оставить как есть или закодировать
        df[col] = df[col].astype(str).fillna('UNKNOWN')
        print(f"   Бинарный признак оставлен как есть")

print(f"\n Обработка категориальных признаков завершена!")
print(f"Создано label encoders: {len(label_encoders)}")
print(f"Создано frequency encodings: {len(frequency_encodings)}")
print(f"Текущий размер данных: {df.shape}")


In [ ]:


# Выбираем только числовые колонки
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
if 'target' in numeric_cols:
    numeric_cols.remove('target')
if 'user_id' in numeric_cols:
    numeric_cols.remove('user_id')

print(f"\nПроверяем {len(numeric_cols)} числовых признаков на выбросы...")

# Метод IQR для обнаружения выбросов
outliers_info = {}
outliers_count = 0

for col in numeric_cols[:20]:  # Проверяем первые 20 для скорости
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    outlier_count = len(outliers)
    
    if outlier_count > 0:
        outliers_info[col] = {
            'count': outlier_count,
            'percentage': (outlier_count / len(df)) * 100,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound
        }
        outliers_count += outlier_count

print(f"\n Обнаружено выбросов: {outliers_count}")
print(f"Колонок с выбросами: {len(outliers_info)}")

# Показываем топ-10 колонок с наибольшим количеством выбросов
if len(outliers_info) > 0:
    sorted_outliers = sorted(outliers_info.items(), key=lambda x: x[1]['count'], reverse=True)[:10]
    print("\nТоп-10 колонок с выбросами:")
    for col, info in sorted_outliers:
        print(f"  {col}: {info['count']} выбросов ({info['percentage']:.2f}%)")

# Решение: Для fraud detection выбросы важны (аномалии = мошенничество)
# Поэтому логируем их





In [ ]:


# Выбираем числовые колонки для масштабирования
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Исключаем колонки, которые НЕ нужно масштабировать:
# - target (целевая переменная)
# - user_id (идентификатор)
# - бинарные признаки (0/1)
# - циклические признаки (sin/cos уже в диапазоне -1 до 1)
exclude_cols = ['target', 'user_id']
exclude_patterns = ['_sin', '_cos', 'is_', '_encoded']  # Циклические и бинарные

cols_to_scale = []
for col in numeric_cols:
    if col in exclude_cols:
        continue
    if any(pattern in col for pattern in exclude_patterns):
        continue
    # Проверяем, что это не бинарный признак (только 0 и 1)
    unique_vals = df[col].dropna().unique()
    if len(unique_vals) <= 2 and set(unique_vals).issubset({0, 1, 0.0, 1.0}):
        continue
    cols_to_scale.append(col)

print(f"\nНайдено {len(cols_to_scale)} признаков для масштабирования")
print(f"Примеры: {cols_to_scale[:5]}")

# Используем MinMaxScaler для масштабирования в диапазон [0, 1]
# Это лучше для tree-based моделей (LightGBM, XGBoost)
scaler = MinMaxScaler()

# Сохраняем исходные данные для масштабирования
df_scaled = df.copy()
df_scaled[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

# Обновляем основной датафрейм
df = df_scaled

print(f"\n Масштабировано {len(cols_to_scale)} признаков с помощью MinMaxScaler")
print("Признаки масштабированы в диапазон [0, 1]")

# Сохраняем scaler для использования в pipeline
print(f"\nScaler сохранен для использования в preprocessing pipeline")


In [ ]:

print("="*80)
print("ЗАДАЧА 32: СОХРАНЕНИЕ ФИНАЛЬНОГО ДАТАСЕТА")
print("="*80)

# Заполняем оставшиеся пропуски
df.fillna(0, inplace=True)

# Убеждаемся, что все колонки числовые (кроме user_id если он нужен)
# Удаляем user_id если он не нужен для модели
if 'user_id' in df.columns:
    # Сохраняем user_id отдельно для возможного использования
    user_ids = df['user_id'].copy()
    df_for_model = df.drop(columns=['user_id'], errors='ignore')
else:
    df_for_model = df.copy()

# Проверяем финальную структуру
print(f"\n Финальная структура данных:")
print(f"  Размер: {df_for_model.shape}")
print(f"  Колонок: {len(df_for_model.columns)}")
print(f"  Строк: {len(df_for_model)}")

# Проверяем наличие target
if 'target' not in df_for_model.columns:
    raise ValueError(" Колонка 'target' отсутствует!")

# Сохраняем финальный датасет
output_dir = os.path.dirname(OUTPUT_PATH)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir, exist_ok=True)

df_for_model.to_csv(OUTPUT_PATH, index=False)
print(f"\n Финальный датасет сохранен: {OUTPUT_PATH}")

# Также сохраняем версию с user_id (если нужна)
if 'user_id' in df.columns:
    df.to_csv(OUTPUT_PATH.replace('_final.csv', '_with_user_id.csv'), index=False)
    print(f" Версия с user_id сохранена: {OUTPUT_PATH.replace('_final.csv', '_with_user_id.csv')}")



In [ ]:


# Создаем класс для preprocessing pipeline
class FraudDetectionPreprocessor:
    """
    Preprocessing pipeline для fraud detection модели.
    Выполняет все необходимые преобразования данных.
    """
    
    def __init__(self, label_encoders=None, frequency_encodings=None, scaler=None, feature_columns=None):
        self.label_encoders = label_encoders or {}
        self.frequency_encodings = frequency_encodings or {}
        self.scaler = scaler
        self.feature_columns = feature_columns  # Список финальных признаков
        
    def fit(self, df):
        """Обучение pipeline на данных"""
        # Сохраняем список финальных признаков (без target и user_id)
        if self.feature_columns is None:
            self.feature_columns = [col for col in df.columns 
                                   if col not in ['target', 'user_id']]
        return self
    
    def transform(self, df):
        """Применение преобразований к данным"""
        df_processed = df.copy()
        
        # 1. Обработка категориальных признаков
        for col, le in self.label_encoders.items():
            if col in df_processed.columns:
                df_processed[f'{col}_encoded'] = le.transform(
                    df_processed[col].astype(str).fillna('UNKNOWN')
                )
                df_processed.drop(columns=[col], inplace=True, errors='ignore')
        
        for col, freq_map in self.frequency_encodings.items():
            if col in df_processed.columns:
                df_processed[f'{col}_freq_encoding'] = df_processed[col].map(freq_map).fillna(0)
                df_processed.drop(columns=[col], inplace=True, errors='ignore')
        
        # 2. Заполнение пропусков
        df_processed.fillna(0, inplace=True)
        
        # 3. Масштабирование (если scaler предоставлен)
        if self.scaler is not None and self.feature_columns is not None:
            available_cols = [col for col in self.feature_columns if col in df_processed.columns]
            if available_cols:
                df_processed[available_cols] = self.scaler.transform(df_processed[available_cols])
        
        # 4. Выбор только нужных признаков
        if self.feature_columns is not None:
            # Сохраняем target и user_id если они есть
            keep_cols = self.feature_columns.copy()
            if 'target' in df_processed.columns:
                keep_cols.append('target')
            if 'user_id' in df_processed.columns:
                keep_cols.append('user_id')
            df_processed = df_processed[[col for col in keep_cols if col in df_processed.columns]]
        
        return df_processed
    
    def fit_transform(self, df):
        """Обучение и применение pipeline"""
        return self.fit(df).transform(df)

# Создаем и сохраняем pipeline
preprocessor = FraudDetectionPreprocessor(
    label_encoders=label_encoders,
    frequency_encodings=frequency_encodings,
    scaler=scaler,
    feature_columns=[col for col in df_for_model.columns if col != 'target']
)

# Сохраняем pipeline
os.makedirs(os.path.dirname(PIPELINE_PATH), exist_ok=True)
joblib.dump(preprocessor, PIPELINE_PATH)
print(f"\n Preprocessing pipeline сохранен: {PIPELINE_PATH}")




In [ ]:


def validate_input_data(df, required_features=None, target_col='target'):
    """
    Валидация входных данных для fraud detection модели.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Входные данные для валидации
    required_features : list, optional
        Список обязательных признаков. Если None, берется из сохраненной модели
    target_col : str, default='target'
        Название колонки с целевой переменной (если есть)
    
    Returns:
    --------
    dict : Словарь с результатами валидации
        - 'is_valid': bool - прошла ли валидация
        - 'errors': list - список ошибок
        - 'warnings': list - список предупреждений
    """
    errors = []
    warnings = []
    
    # 1. Проверка типа данных
    if not isinstance(df, pd.DataFrame):
        errors.append("Входные данные должны быть pandas.DataFrame")
        return {'is_valid': False, 'errors': errors, 'warnings': warnings}
    
    # 2. Проверка наличия данных
    if df.empty:
        errors.append("DataFrame пуст")
        return {'is_valid': False, 'errors': errors, 'warnings': warnings}
    
    # 3. Проверка обязательных признаков
    if required_features is not None:
        missing_features = [f for f in required_features if f not in df.columns]
        if missing_features:
            errors.append(f"Отсутствуют обязательные признаки: {missing_features[:10]}")
    
    # 4. Проверка типов данных (все должны быть числовыми, кроме target и user_id)
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    non_numeric_cols = [col for col in df.columns 
                       if col not in numeric_cols and col not in [target_col, 'user_id']]
    if non_numeric_cols:
        errors.append(f"Найдены нечисловые колонки (кроме target/user_id): {non_numeric_cols[:5]}")
    
    # 5. Проверка пропущенных значений
    missing_counts = df.isnull().sum()
    cols_with_missing = missing_counts[missing_counts > 0]
    if len(cols_with_missing) > 0:
        warnings.append(f"Найдены пропущенные значения в {len(cols_with_missing)} колонках")
        warnings.append(f"Примеры: {dict(cols_with_missing.head(5))}")
    
    # 6. Проверка бесконечных значений
    inf_counts = np.isinf(df.select_dtypes(include=[np.number])).sum()
    cols_with_inf = inf_counts[inf_counts > 0]
    if len(cols_with_inf) > 0:
        errors.append(f"Найдены бесконечные значения в {len(cols_with_inf)} колонках")
    
    # 7. Проверка размера данных
    if len(df) == 0:
        errors.append("Нет данных для обработки")
    elif len(df) > 1000000:
        warnings.append(f"Большой объем данных: {len(df)} строк. Обработка может занять время")
    
    is_valid = len(errors) == 0
    
    return {
        'is_valid': is_valid,
        'errors': errors,
        'warnings': warnings,
        'shape': df.shape,
        'numeric_cols_count': len(numeric_cols),
        'missing_values_count': missing_counts.sum()
    }

# Тестируем функцию валидации на наших данных
print("\n--- Тестирование функции валидации ---")
validation_result = validate_input_data(df_for_model, 
                                        required_features=[col for col in df_for_model.columns if col != 'target'])

print(f"\nРезультат валидации:")
print(f"  Валидно: {validation_result['is_valid']}")
print(f"  Ошибок: {len(validation_result['errors'])}")
print(f"  Предупреждений: {len(validation_result['warnings'])}")

if validation_result['errors']:
    print(f"\nОшибки:")
    for error in validation_result['errors']:
        print(f"   {error}")

if validation_result['warnings']:
    print(f"\nПредупреждения:")
    for warning in validation_result['warnings']:
        print(f"   {warning}")

# Сохраняем функцию валидации в отдельный файл для использования в API
validation_code = '''
def validate_input_data(df, required_features=None, target_col='target'):
    """Валидация входных данных для fraud detection модели."""
    import pandas as pd
    import numpy as np
    
    errors = []
    warnings = []
    
    if not isinstance(df, pd.DataFrame):
        errors.append("Входные данные должны быть pandas.DataFrame")
        return {'is_valid': False, 'errors': errors, 'warnings': warnings}
    
    if df.empty:
        errors.append("DataFrame пуст")
        return {'is_valid': False, 'errors': errors, 'warnings': warnings}
    
    if required_features is not None:
        missing_features = [f for f in required_features if f not in df.columns]
        if missing_features:
            errors.append(f"Отсутствуют обязательные признаки: {missing_features[:10]}")
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    non_numeric_cols = [col for col in df.columns 
                       if col not in numeric_cols and col not in [target_col, 'user_id']]
    if non_numeric_cols:
        errors.append(f"Найдены нечисловые колонки: {non_numeric_cols[:5]}")
    
    missing_counts = df.isnull().sum()
    cols_with_missing = missing_counts[missing_counts > 0]
    if len(cols_with_missing) > 0:
        warnings.append(f"Найдены пропущенные значения в {len(cols_with_missing)} колонках")
    
    inf_counts = np.isinf(df.select_dtypes(include=[np.number])).sum()
    cols_with_inf = inf_counts[inf_counts > 0]
    if len(cols_with_inf) > 0:
        errors.append(f"Найдены бесконечные значения в {len(cols_with_inf)} колонках")
    
    is_valid = len(errors) == 0
    
    return {
        'is_valid': is_valid,
        'errors': errors,
        'warnings': warnings,
        'shape': df.shape
    }
'''

# Сохраняем функцию в файл
validation_file_path = '../src/data_validation.py'
os.makedirs(os.path.dirname(validation_file_path), exist_ok=True)
with open(validation_file_path, 'w', encoding='utf-8') as f:
    f.write(validation_code)

print(f"\n Функция валидации сохранена: {validation_file_path}")



In [ ]:


# Получаем финальный список признаков (без target)
final_features = [col for col in df_for_model.columns if col != 'target']

print(f"\n ФИНАЛЬНЫЙ НАБОР ПРИЗНАКОВ:")
print(f"  Всего признаков: {len(final_features)}")
print(f"  Размер данных: {df_for_model.shape}")

# Группируем признаки по категориям
feature_categories = {
    'Временные': [f for f in final_features if any(x in f for x in ['hour', 'day', 'month', 'weekend', 'night', 'sin', 'cos'])],
    'По пользователю': [f for f in final_features if 'user_' in f],
    'Rolling window': [f for f in final_features if any(x in f for x in ['_1h', '_12h', '_24h'])],
    'Статистические': [f for f in final_features if any(x in f for x in ['zscore', 'percentile', 'ratio', 'cv'])],
    'Device/OS': [f for f in final_features if any(x in f for x in ['device', 'os'])],
    'Поведенческие': [f for f in final_features if any(x in f for x in ['time_since', 'time_until', 'tx_rate', 'interval', 'rapid', 'anomaly'])],
    'Поведенческие (исходные)': [f for f in final_features if 'количество' in f or 'среднее' in f or 'доля' in f or 'интервал' in f],
    'Прочие': []
}

# Распределяем оставшиеся признаки
categorized = set()
for category, features in feature_categories.items():
    categorized.update(features)

feature_categories['Прочие'] = [f for f in final_features if f not in categorized]

print(f"\n Распределение признаков по категориям:")
for category, features in feature_categories.items():
    if features:
        print(f"  {category}: {len(features)} признаков")
        if len(features) <= 5:
            print(f"    Примеры: {features}")
        else:
            print(f"    Примеры: {features[:3]} ... и еще {len(features)-3}")

# Сохраняем список финальных признаков
features_list_path = '../model/final_features_list.txt'
with open(features_list_path, 'w', encoding='utf-8') as f:
    f.write("ФИНАЛЬНЫЙ НАБОР ПРИЗНАКОВ ДЛЯ ML МОДЕЛИ\n")
    f.write("="*80 + "\n\n")
    f.write(f"Всего признаков: {len(final_features)}\n\n")
    for category, features in feature_categories.items():
        if features:
            f.write(f"{category} ({len(features)} признаков):\n")
            for feat in features:
                f.write(f"  - {feat}\n")
            f.write("\n")

print(f"\n Список признаков сохранен: {features_list_path}")

# Сохраняем также в pickle для удобства
import joblib
joblib.dump(final_features, '../model/final_features.pkl')
print(f" Список признаков сохранен (pickle): ../model/final_features.pkl")




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
import os
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)




## Загрузка данных с признаками


In [ ]:
# Загрузка финального датасета с признаками
FEATURES_FILE = '../data/processed/transactions_with_features.csv'

try:
    df = pd.read_csv(FEATURES_FILE)
    print(f" Данные загружены: {df.shape}")
    print(f"Колонок: {len(df.columns)}")
    print(f"Строк: {len(df)}")
except FileNotFoundError:
    print(f" Файл {FEATURES_FILE} не найден!")
    raise


In [ ]:

categorical_cols = []
for col in df.columns:
    if col in ['target', 'user_id']:
        continue
    if df[col].dtype == 'object':
        categorical_cols.append(col)
    elif df[col].dtype in [np.int64, np.int32] and df[col].nunique() < 20:
        # Целочисленные колонки с небольшим количеством уникальных значений тоже могут быть категориальными
        if col not in ['hour', 'day_of_week', 'day_of_month', 'month']:
            categorical_cols.append(col)

print(f"Найдено категориальных признаков: {len(categorical_cols)}")
if len(categorical_cols) > 0:
    print("Категориальные колонки:")
    for i, col in enumerate(categorical_cols[:10], 1):
        unique_count = df[col].nunique()
        print(f"  {i}. {col} ({unique_count} уникальных значений)")
    if len(categorical_cols) > 10:
        print(f"  ... и еще {len(categorical_cols) - 10} колонок")

# Обработка категориальных признаков
processed_categorical = []

for col in categorical_cols:
    if col not in df.columns:
        continue
    
    # Заполняем пропуски
    df[col].fillna('UNKNOWN', inplace=True)
    
    unique_count = df[col].nunique()
    
    # Frequency Encoding (для всех категориальных)
    freq_counts = df[col].value_counts()
    df[f'{col}_freq'] = df[col].map(freq_counts) / len(df)
    processed_categorical.append(f'{col}_freq')
    
    # Target Encoding (если есть target) - среднее значение target для каждой категории
    if 'target' in df.columns:
        target_mean = df.groupby(col)['target'].mean()
        df[f'{col}_target_enc'] = df[col].map(target_mean)
        df[f'{col}_target_enc'].fillna(df['target'].mean(), inplace=True)  # Для новых категорий
        processed_categorical.append(f'{col}_target_enc')
    
    # Удаляем исходную категориальную колонку (после создания encoding)
    if unique_count > 10:  # Удаляем только если много уникальных значений
        df.drop(columns=[col], inplace=True, errors='ignore')
        print(f" Обработан {col}: создан frequency encoding, удалена исходная колонка")
    else:
        print(f" Обработан {col}: создан frequency encoding, исходная колонка сохранена")

print(f"\n Обработано {len(categorical_cols)} категориальных признаков")
print(f"Создано {len(processed_categorical)} новых признаков из категориальных")


In [ ]:


# Выбираем только числовые колонки для анализа выбросов
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['target', 'user_id']]

print(f"Анализ выбросов для {len(numeric_cols)} числовых признаков\n")

# Методы обнаружения выбросов
outliers_info = []

for col in numeric_cols[:20]:  # Анализируем первые 20 признаков (чтобы не перегружать)
    if col not in df.columns:
        continue
    
    # Метод IQR (Interquartile Range)
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers_iqr = ((df[col] < lower_bound) | (df[col] > upper_bound)).sum()
    outliers_pct = (outliers_iqr / len(df)) * 100
    
    # Метод Z-score (выбросы > 3 стандартных отклонений)
    z_scores = np.abs((df[col] - df[col].mean()) / (df[col].std() + 1e-8))
    outliers_zscore = (z_scores > 3).sum()
    outliers_zscore_pct = (outliers_zscore / len(df)) * 100
    
    if outliers_iqr > 0 or outliers_zscore > 0:
        outliers_info.append({
            'column': col,
            'outliers_iqr': outliers_iqr,
            'outliers_iqr_pct': outliers_pct,
            'outliers_zscore': outliers_zscore,
            'outliers_zscore_pct': outliers_zscore_pct
        })

if len(outliers_info) > 0:
    outliers_df = pd.DataFrame(outliers_info)
    outliers_df = outliers_df.sort_values('outliers_iqr', ascending=False)
    
    print("ТОП-10 признаков с наибольшим количеством выбросов (IQR метод):")
    print("=" * 80)
    print(outliers_df.head(10).to_string(index=False))
    
    # Визуализация выбросов для топ-5 признаков
    top_5_cols = outliers_df.head(5)['column'].tolist()
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    axes = axes.flatten()
    
    for i, col in enumerate(top_5_cols[:6]):
        if col in df.columns:
            # Box plot для визуализации выбросов
            axes[i].boxplot(df[col].dropna(), vert=True, patch_artist=True,
                          boxprops=dict(facecolor='lightblue', alpha=0.7))
            axes[i].set_title(f'{col[:40]}...\nВыбросов: {outliers_df[outliers_df["column"]==col]["outliers_iqr"].values[0]:.0f}', 
                           fontsize=10, fontweight='bold')
            axes[i].set_ylabel('Значение')
            axes[i].grid(alpha=0.3, axis='y')
    
    # Скрываем лишние subplot'ы
    for i in range(len(top_5_cols), 6):
        axes[i].axis('off')
    
    plt.suptitle('Визуализация выбросов (топ-5 признаков)', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    print("\n⚠️ ВНИМАНИЕ: Выбросы могут быть важными для обнаружения мошенничества!")
    print("   Рекомендуется НЕ удалять выбросы, а использовать модели, устойчивые к ним")
    print("   (например, Random Forest, XGBoost, LightGBM)")
else:
    print(" Выбросов не обнаружено (или все признаки уже обработаны)")

print("\n Проверка выбросов завершена")


In [ ]:

# Проверяем, какие признаки уже масштабированы (должны быть в диапазоне [0, 1] или около того)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ['target', 'user_id']]

# Исключаем бинарные и циклические признаки
cols_to_check = [col for col in numeric_cols 
                 if not any(x in col.lower() for x in ['_sin', '_cos', 'is_', 'changed', 'hour', 'day_of_week', 'day_of_month', 'month'])]

print(f"Проверка масштабирования для {len(cols_to_check)} признаков\n")

# Проверяем диапазоны значений
scaling_status = []
for col in cols_to_check[:15]:  # Проверяем первые 15
    if col in df.columns:
        min_val = df[col].min()
        max_val = df[col].max()
        mean_val = df[col].mean()
        
        # Если значения в диапазоне [0, 1] или близко к нему - вероятно масштабировано
        is_scaled = (min_val >= -0.1 and max_val <= 1.1) or (abs(mean_val) < 1 and abs(min_val) < 10 and abs(max_val) < 10)
        
        scaling_status.append({
            'column': col,
            'min': min_val,
            'max': max_val,
            'mean': mean_val,
            'likely_scaled': is_scaled
        })

if len(scaling_status) > 0:
    scaling_df = pd.DataFrame(scaling_status)
    print("Статус масштабирования признаков:")
    print("=" * 80)
    print(scaling_df.to_string(index=False))
    
    scaled_count = scaling_df['likely_scaled'].sum()
    print(f"\n Вероятно масштабировано: {scaled_count} из {len(scaling_df)} проверенных признаков")
else:
    print(" Не удалось проверить масштабирование")



In [ ]:


FEATURES_OUTPUT = '../data/processed/transactions_with_features.csv'

if os.path.exists(FEATURES_OUTPUT):
    # Проверяем размер файла
    file_size = os.path.getsize(FEATURES_OUTPUT) / (1024 * 1024)  # в MB
    
    print(f"   Файл features.csv существует: {FEATURES_OUTPUT}")
    print(f"   Размер файла: {file_size:.2f} MB")
    print(f"   Размер данных: {df.shape}")
    print(f"   Количество признаков (без target): {len([c for c in df.columns if c != 'target'])}")
    
    # Проверяем наличие target
    if 'target' in df.columns:
        target_dist = df['target'].value_counts()
        print(f"\n   Распределение target:")
        for val, count in target_dist.items():
            print(f"     Класс {val}: {count} ({count/len(df)*100:.2f}%)")
    
    print(f"\n Файл features.csv готов!")
else:
    print(f" Файл {FEATURES_OUTPUT} не найден!")
    
    
    output_dir = os.path.dirname(FEATURES_OUTPUT)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    df.to_csv(FEATURES_OUTPUT, index=False)
    print(f" Файл сохранен: {FEATURES_OUTPUT}")


In [ ]:


# Создаем функцию для preprocessing pipeline

def create_preprocessing_pipeline(df_train, df_test=None):

    
    # Копируем данные
    train = df_train.copy()
    
    # Разделяем на признаки и целевую переменную
    if 'target' in train.columns:
        y_train = train['target'].copy()
        X_train = train.drop(columns=['target'], errors='ignore')
    else:
        y_train = None
        X_train = train.copy()
    
    # Удаляем служебные колонки
    cols_to_drop = ['user_id', 'timestamp'] if 'user_id' in X_train.columns else []
    X_train = X_train.drop(columns=cols_to_drop, errors='ignore')
    
    # Заполняем пропуски
    X_train = X_train.fillna(0)
    
    # Обработка тестового датасета (если передан)
    if df_test is not None:
        test = df_test.copy()
        
        if 'target' in test.columns:
            y_test = test['target'].copy()
            X_test = test.drop(columns=['target'], errors='ignore')
        else:
            y_test = None
            X_test = test.copy()
        
        X_test = X_test.drop(columns=cols_to_drop, errors='ignore')
        X_test = X_test.fillna(0)
        
        # Убеждаемся, что колонки совпадают
        missing_cols = set(X_train.columns) - set(X_test.columns)
        if missing_cols:
            for col in missing_cols:
                X_test[col] = 0
        
        X_test = X_test[X_train.columns]  # Приводим к тому же порядку
        
        return X_train, y_train, X_test, y_test
    
    return X_train, y_train



# Тестируем pipeline на текущих данных
if 'target' in df.columns:
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['target'])
    
    X_train, y_train, X_test, y_test = create_preprocessing_pipeline(train_df, test_df)
    
    print(f"\n Pipeline протестирован:")
    print(f"   X_train shape: {X_train.shape}")
    print(f"   X_test shape: {X_test.shape}")
    print(f"   y_train distribution: {y_train.value_counts().to_dict()}")
    print(f"   y_test distribution: {y_test.value_counts().to_dict()}")
    print(f"\n Preprocessing pipeline готов к использованию!")
else:
    print("\n Колонка 'target' не найдена, pipeline не протестирован")



In [ ]:


# Получаем список всех признаков (без target и user_id)
feature_cols = [col for col in df.columns if col not in ['target', 'user_id']]

print(f" ФИНАЛЬНЫЙ НАБОР ПРИЗНАКОВ: {len(feature_cols)} признаков\n")
print("=" * 80)

# Группируем признаки по категориям
temporal_features = [col for col in feature_cols if any(x in col for x in ['hour', 'day', 'month', 'weekend', 'night', '_sin', '_cos'])]
user_features = [col for col in feature_cols if 'user_' in col or 'amount_diff' in col or 'amount_ratio' in col]
rolling_features = [col for col in feature_cols if any(x in col for x in ['tx_count', 'tx_mean', 'tx_std', 'tx_sum', 'tx_max'])]
statistical_features = [col for col in feature_cols if any(x in col for x in ['zscore', 'percentile', 'ratio', 'cv_amount'])]
device_features = [col for col in feature_cols if any(x in col for x in ['device', 'os', 'ip', 'geo'])]
behavioral_features = [col for col in feature_cols if any(x in col for x in ['time_since', 'time_until', 'rate', 'interval', 'rapid', 'anomaly', 'morning', 'afternoon', 'evening'])]
categorical_features = [col for col in feature_cols if '_freq' in col or '_target_enc' in col]
other_features = [col for col in feature_cols if col not in temporal_features + user_features + rolling_features + statistical_features + device_features + behavioral_features + categorical_features]

categories = {
    'Временные признаки': temporal_features,
    'Признаки по пользователю': user_features,
    'Rolling-window признаки': rolling_features,
    'Статистические признаки': statistical_features,
    'Device/OS/IP признаки': device_features,
    'Признаки поведения': behavioral_features,
    'Категориальные encoding': categorical_features,
    'Другие признаки': other_features
}

# Выводим статистику по категориям
print("Распределение признаков по категориям:\n")
for category, cols in categories.items():
    if len(cols) > 0:
        print(f"  {category}: {len(cols)} признаков")

print(f"\n  ВСЕГО: {len(feature_cols)} признаков")

# Сохраняем список признаков в файл
features_list_file = '../data/processed/features_list.txt'
with open(features_list_file, 'w', encoding='utf-8') as f:
    f.write("ФИНАЛЬНЫЙ НАБОР ПРИЗНАКОВ ДЛЯ ML МОДЕЛИ\n")
    f.write("=" * 80 + "\n\n")
    f.write(f"Всего признаков: {len(feature_cols)}\n\n")
    
    for category, cols in categories.items():
        if len(cols) > 0:
            f.write(f"{category} ({len(cols)}):\n")
            for col in cols:
                f.write(f"  - {col}\n")
            f.write("\n")

print(f"\n Список признаков сохранен в: {features_list_file}")

# Выводим первые 20 признаков для примера
print("\nПримеры признаков (первые 20):")
for i, col in enumerate(feature_cols[:20], 1):
    print(f"  {i:2d}. {col}")
if len(feature_cols) > 20:
    print(f"  ... и еще {len(feature_cols) - 20} признаков")

print(f"\n Финальный набор из {len(feature_cols)} признаков готов для ML модели!")


In [ ]:


print("\n ФИНАЛЬНАЯ СТАТИСТИКА:")
print(f"  - Всего признаков: {len([c for c in df.columns if c != 'target'])}")
print(f"  - Размер датасета: {df.shape}")
if 'target' in df.columns:
    target_dist = df['target'].value_counts()
    print(f"  - Распределение классов: {target_dist.to_dict()}")


